In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

plt.ion()   # interactive mode
%matplotlib inline

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

train_loader = torch.utils.data.DataLoader(datasets.MNIST(root='data/', train=True, download=True,
                                                          transform=transforms.Compose([
                                                              transforms.ToTensor(),
                                                              transforms.Normalize((0.1307,),(0.3081,))
                                                          ])), batch_size = 64, shuffle = True, num_workers = 4)

test_loader = torch.utils.data.DataLoader(datasets.MNIST(root='data/', train=False, 
                                                         transform=transforms.Compose([
                                                             transforms.ToTensor(),
                                                             transforms.Normalize((0.1307,), (0.3081,))
                                                         ])), batch_size=64, shuffle=True, num_workers=4)


In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
        self.localization = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=7),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True),
            nn.Conv2d(8, 10, kernel_size=5),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True)
        )
        
        self.fc_loc = nn.Sequential(
            nn.Linear(10 * 3 * 3, 32),
            nn.ReLU(True),
            nn.Linear(32, 3 * 2)
        )
        
        self.fc_loc[2].weight.data.zero_()
        self.fc_loc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))
        
    def stn(self, x):
        xs = self.localization(x)
        xs = xs.view(-1, 10 * 3 * 3)
        theta = self.fc_loc(xs)
        theta = theta.view(-1, 2, 3)
        
        grid = F.affine_grid(theta, x.size())
        x = F.grid_sample(x, grid)
        
        return x
    
    def forward(self, x):
        
        x = self.stn(x)
        
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


model = Net().to(device)

In [7]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 500 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test():
    with torch.no_grad():
        model.eval()
        test_loss = 0
        correct = 0
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            test_loss += F.nll_loss(output, target, size_average=False).item()
            
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
        test_loss /= len(test_loader.dataset)
        print('\n Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'
              .format(test_loss, correct, len(test_loader.dataset),
                      100. * correct/len(test_loader.dataset)))

In [ ]:
def convert_image_np(inp):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    return inp

def visualize_stn():
    with torch.no_grad():
        data = next(iter(test_loader))[0].to(device)
        
        input_tensor = data.cpu()
        transformed_input_tensor = model.stn(data).cpu()
        
        in_grid = convert_image_np(
            torchvision.utils.make_grid(input_tensor))
        
        out_grid = convert_image_np(
            torchvision.utils.make_grid(transformed_input_tensor))
        
        f, axarr = plt.subplots(1, 2)
        axarr[0].imshow(in_grid)
        axarr[0].set_title('Dataset Images')
        
        axarr[1].imshow(out_grid)
        axarr[1].set_title('Transofrmed Images')
        
for epoch in range(1, 20 + 1):
    train(epoch)
    test()
    
visualize_stn()
plt.ioff()
plt.show()

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.332689
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.170960

 Test set: Average loss: 0.9057, Accuracy: 7362/10000 (74%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.886189
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.326185

 Test set: Average loss: 0.0838, Accuracy: 9733/10000 (97%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.394539
Train Epoch: 3 [32000/60000 (53%)]	Loss: 0.419149

 Test set: Average loss: 0.0733, Accuracy: 9760/10000 (98%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.168806
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.185782

 Test set: Average loss: 0.0731, Accuracy: 9760/10000 (98%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.311886
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.204089

 Test set: Average loss: 0.0646, Accuracy: 9799/10000 (98%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.241806
Train Epoch: 6 [32000/60000 (53%)]	Loss: 0.179459

 Test set: Average loss: 0.0568, Accuracy: 9826/10000 (98%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.193199
Train Epoch: 